# 253.1.[실습] CNN활용_폴더에서 데이터수집_알츠하이머_MRI 분류

In [2]:
# 출처: 모두의 딥러닝, 개정2판, 제20장

### 실습: 치매 환자의 뇌인지 일반인의 뇌인지 예측하기

## 1. 전이학습 없는 CNN 구현(ImageDataGenerator 활용- 데이터 증강 없음)

In [4]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
from keras import optimizers
# from charset_normalizer import md__mypyc
import numpy as np
import matplotlib.pyplot as plt

#학습셋의 변형을 설정하는 부분입니다. 
train_datagen = ImageDataGenerator(rescale=1./255,          # 주어진 이미지의 크기를 설정합니다.
                                   horizontal_flip=True,     # 수평 대칭 이미지를 50% 확률로 만들어 추가합니다.
                                   width_shift_range=0.1,    # 전체 크기의 15% 범위에서 좌우로 이동합니다.
                                   height_shift_range=0.1,   # 마찬가지로 위, 아래로 이동합니다.
                                   #rotation_range=5,        # 정해진 각도만큼 회전시킵니다.
                                   #shear_range=0.7,         # 좌표 하나를 고정시키고 나머지를 이동시킵니다.
                                   #zoom_range=1.2,          # 확대 또는 축소시킵니다.
                                   #vertical_flip=True,      # 수직 대칭 이미지를 만듭니다.
                                   #fill_mode='nearest'      # 빈 공간을 채우는 방법입니다. nearest 옵션은 가장 비슷한 색으로 채우게 됩니다.
                                   )      

train_generator = train_datagen.flow_from_directory(
       'data/alzheimer_data/alzheimer_data/train',   # 학습셋이 있는 폴더의 위치입니다.
       target_size=(150, 150),
       batch_size=5,
       class_mode='binary')

# 테스트셋은 이미지 부풀리기 과정을 진행하지 않습니다.
test_datagen = ImageDataGenerator(rescale=1./255)  

test_generator = test_datagen.flow_from_directory(
       'data/alzheimer_data/alzheimer_data/test',   # 테스트셋이 있는 폴더의 위치입니다.
       target_size=(150, 150),
       batch_size=5,
       class_mode='binary')

Found 160 images belonging to 2 classes.
Found 120 images belonging to 2 classes.


In [5]:
# 앞서 배운 CNN 모델을 만들어 적용해 보겠습니다.
next(train_generator)

(array([[[[0.        , 0.        , 0.        ],
          [0.        , 0.        , 0.        ],
          [0.        , 0.        , 0.        ],
          ...,
          [0.08627451, 0.08627451, 0.08627451],
          [0.08219118, 0.08219118, 0.08219118],
          [0.07843138, 0.07843138, 0.07843138]],
 
         [[0.        , 0.        , 0.        ],
          [0.        , 0.        , 0.        ],
          [0.        , 0.        , 0.        ],
          ...,
          [0.08526628, 0.08526628, 0.08526628],
          [0.08118296, 0.08118296, 0.08118296],
          [0.07794806, 0.07794806, 0.07794806]],
 
         [[0.        , 0.        , 0.        ],
          [0.        , 0.        , 0.        ],
          [0.        , 0.        , 0.        ],
          ...,
          [0.08235294, 0.08235294, 0.08235294],
          [0.07879452, 0.07879452, 0.07879452],
          [0.0775597 , 0.0775597 , 0.0775597 ]],
 
         ...,
 
         [[0.        , 0.        , 0.        ],
          [0.     

In [6]:
# CNN 모델 구현
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(150,150,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 activation (Activation)     (None, 148, 148, 32)      0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 72, 72, 32)        0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 32)       0         
 2D)                                                    

In [7]:
#모델 최적화
model.compile(optimizer = 'adam',
            loss = 'binary_crossentropy',
            metrics = ['accuracy'])

# 조기 중단을 설정
early_stopping = EarlyStopping(monitor = 'accuracy', patience = 10)

#모델 학습
hist = model.fit(train_generator, 
                epochs = 10,
                # steps_per_epoch  = 50,
                validation_data = test_generator,
                # validation_steps = 1000,
                callbacks = [early_stopping],
                verbose = 2)


Epoch 1/10
32/32 - 6s - loss: 0.7283 - accuracy: 0.4875 - val_loss: 0.6933 - val_accuracy: 0.5000 - 6s/epoch - 186ms/step
Epoch 2/10
32/32 - 3s - loss: 0.6946 - accuracy: 0.5312 - val_loss: 0.6911 - val_accuracy: 0.6167 - 3s/epoch - 92ms/step
Epoch 3/10
32/32 - 3s - loss: 0.6942 - accuracy: 0.4875 - val_loss: 0.6896 - val_accuracy: 0.5583 - 3s/epoch - 98ms/step
Epoch 4/10
32/32 - 3s - loss: 0.6868 - accuracy: 0.5750 - val_loss: 0.6892 - val_accuracy: 0.5000 - 3s/epoch - 85ms/step
Epoch 5/10
32/32 - 3s - loss: 0.6884 - accuracy: 0.5437 - val_loss: 0.6774 - val_accuracy: 0.5917 - 3s/epoch - 92ms/step
Epoch 6/10
32/32 - 3s - loss: 0.6949 - accuracy: 0.5063 - val_loss: 0.6829 - val_accuracy: 0.5000 - 3s/epoch - 92ms/step
Epoch 7/10
32/32 - 3s - loss: 0.6727 - accuracy: 0.5750 - val_loss: 0.6422 - val_accuracy: 0.6000 - 3s/epoch - 94ms/step
Epoch 8/10
32/32 - 3s - loss: 0.6704 - accuracy: 0.6125 - val_loss: 0.5978 - val_accuracy: 0.7000 - 3s/epoch - 94ms/step
Epoch 9/10
32/32 - 3s - loss: 0

In [ ]:
# 검증셋과 학습셋의 오차 저장

# 그래프로 시각화
